<a href="https://colab.research.google.com/github/matrasakti/Recruitment-Prediction-Project/blob/main/Deployment_Recruitment_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle

# Load the Random Forest model from the file
with open('RF_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def agecategorization(df):
    df['New_Age_Categorization'] = pd.cut(
        x=df['Age'],
        bins=[0, 27, 43, 59],
        labels=["Gen Z", "Millenial", "Gen X"]
    )

def interviewcategorization(df):
    df['New_InterviewScore_Categorization'] = pd.cut(
        x=df['InterviewScore'],
        bins=[-1, 21, 41, 62, 79, 100],
        labels=["Very Low", "Low", "Medium", "High", "Very High"]
    )

def skillcategorization(df):
    df['New_SkillScore_Categorization'] = pd.cut(
        x=df['SkillScore'],
        bins=[-1, 20, 42, 62, 80, 100],
        labels=["Very Low", "Low", "Medium", "High", "Very High"]
    )

def personalitycategorization(df):
    df['New_PersonalityScore_Categorization'] = pd.cut(
        x=df['PersonalityScore'],
        bins=[-1, 19, 39, 59, 80, 100],
        labels=["Very Low", "Low", "Medium", "High", "Very High"]
    )

# Function to one-hot encode categorized columns
def one_hot_encode(df):
    # One-hot encode categorical columns
    df = pd.get_dummies(df, columns=[
        'New_Age_Categorization',
        'New_InterviewScore_Categorization',
        'New_SkillScore_Categorization',
        'New_PersonalityScore_Categorization'
    ])
    return df

# Function to scale specific numerical columns
def scale_numerical_columns(df):
    # Columns to scale
    columns_to_scale = ['Age', 'ExperienceYears', 'DistanceFromCompany', 'InterviewScore', 'SkillScore', 'PersonalityScore']

    # Initialize scaler
    scaler = StandardScaler()

    # Fit and transform the specified columns
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

    return df


In [ ]:
def preprocess_input(data):
    df = pd.DataFrame([data])

    # Apply categorization
    agecategorization(df)
    interviewcategorization(df)
    skillcategorization(df)
    personalitycategorization(df)

    # One-hot encode the categorical columns
    df = one_hot_encode(df)

    # Scale the numerical columns
    df = scale_numerical_columns(df)

    # Ensure the DataFrame contains only the required columns
    required_columns = [
        'Age', 'Gender', 'EducationLevel', 'ExperienceYears', 'PreviousCompanies',
        'DistanceFromCompany', 'InterviewScore', 'SkillScore', 'PersonalityScore',
        'RecruitmentStrategy',
        'New_Age_Categorization_Millenial', 'New_Age_Categorization_Gen X',
        'New_InterviewScore_Categorization_Low', 'New_InterviewScore_Categorization_Medium',
        'New_InterviewScore_Categorization_High', 'New_InterviewScore_Categorization_Very High',
        'New_SkillScore_Categorization_Low', 'New_SkillScore_Categorization_Medium',
        'New_SkillScore_Categorization_High', 'New_SkillScore_Categorization_Very High',
        'New_PersonalityScore_Categorization_Low', 'New_PersonalityScore_Categorization_Medium',
        'New_PersonalityScore_Categorization_High', 'New_PersonalityScore_Categorization_Very High'
    ]

    # Add missing columns that weren't created during one-hot encoding
    for col in required_columns:
        if col not in df.columns:
            df[col] = 0

    # Ensure the final DataFrame contains only the required columns
    df = df[required_columns]

    return df


In [ ]:
from flask import Flask, request, render_template_string
import pandas as pd

app = Flask(__name__)

# HTML template for the input form
html_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>Recruitment Model</title>
</head>
<body>
    <h2>Recruitment Probability Prediction</h2>
    <form action="/predict" method="post">
        <label for="age">Age:</label><br>
        <input type="number" id="age" name="age" required><br><br>

        <label for="gender">Gender (1=Male, 0=Female):</label><br>
        <input type="number" id="gender" name="gender" required><br><br>

        <label for="education">Education Level:</label><br>
        <input type="number" id="education" name="education" required><br><br>

        <label for="experience">Years of Experience:</label><br>
        <input type="number" id="experience" name="experience" required><br><br>

        <label for="previous_companies">Previous Companies:</label><br>
        <input type="number" id="previous_companies" name="previous_companies" required><br><br>

        <label for="distance">Distance from Company:</label><br>
        <input type="number" id="distance" name="distance" required><br><br>

        <label for="interview_score">Interview Score:</label><br>
        <input type="number" id="interview_score" name="interview_score" required><br><br>

        <label for="skill_score">Skill Score:</label><br>
        <input type="number" id="skill_score" name="skill_score" required><br><br>

        <label for="personality_score">Personality Score:</label><br>
        <input type="number" id="personality_score" name="personality_score" required><br><br>

        <label for="strategy">Recruitment Strategy (numeric):</label><br>
        <input type="number" id="strategy" name="strategy" required><br><br>

        <input type="submit" value="Predict">
    </form>
    <br>
    {% if prediction %}
    <h3>{{ prediction }}</h3>
    {% endif %}
</body>
</html>
'''

# Route for the home page (input form)
@app.route('/', methods=['GET'])
def home():
    return render_template_string(html_template)

# Route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Collect form data
        input_data = {
            'Age': float(request.form['age']),
            'Gender': float(request.form['gender']),
            'EducationLevel': float(request.form['education']),
            'ExperienceYears': float(request.form['experience']),
            'PreviousCompanies': float(request.form['previous_companies']),
            'DistanceFromCompany': float(request.form['distance']),
            'InterviewScore': float(request.form['interview_score']),
            'SkillScore': float(request.form['skill_score']),
            'PersonalityScore': float(request.form['personality_score']),
            'RecruitmentStrategy': float(request.form['strategy'])
        }

        # Preprocess input
        processed_input = preprocess_input(input_data)

        # Convert to numpy array
        input_array = processed_input.values

        # Predict using the model
        prediction = model.predict_proba(input_array)[0][1]  # Probability of acceptance
        probability_percentage = round(prediction * 100, 2)

        # Render the prediction
        return render_template_string(html_template, prediction=f'Acceptance Probability: {probability_percentage}%')

    except Exception as e:
        return render_template_string(html_template, prediction=f'Error: {str(e)}')


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

!ngrok authtoken 2jjsQAjKw0oZLUlQfIVefQil90V_4QLCeymJrNVih1gEVZFhQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from threading import Thread

def run_app():
    app.run(port=5000)

# Start the Flask app
thread = Thread(target=run_app)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
# Open an ngrok tunnel to the default Flask port 5000
public_url = ngrok.connect(5000).public_url
print(f'Public URL: {public_url}')

Public URL: https://48e3-35-196-120-83.ngrok-free.app
